# Critic Agents and Tree Search

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1A2id3IyP1tSQXmtLsaY9-zyowSRzTaxk?usp=sharing)

⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)

## Philosophical Bits

*What magical trick makes us intelligent? The trick is that there is no trick. The power of intelligence stems from our vast diversity, not from any single, perfect principle.*

-- Marvin Minsky, The Society of Mind, p. 308

In this section, we will take a spite of the task-oriented `RolyPlaying()` class. We design this in an instruction-following manner. The essence is that to solve a complex task, you can enable two communicative agents collabratively working together step by step to reach solutions. The main concepts include:
- **Task**: a task can be as simple as an idea, initialized by an inception prompt.
- **AI User**: the agent who is expected to provide instructions.
- **AI Assistant**: the agent who is expected to respond with solutions that fulfills the instructions. 

**Prerequisite**: We assume that you have read the section on [intro to role-playing](https://colab.research.google.com/drive/1cmWPxXEsyMbmjPhD2bWfHuhd_Uz6FaJQ?usp=sharing).

How do agents accomplish hard tasks? While reasoning can naturally emerge from next-token-prediction pretraining, it is still difficult for agents to solve complex tasks which require lots of intermediate steps. To tackle this issue, tree search is a simple and effective framework.

A typical tree search include node expansion and node selection. In the [March 2023 paper](https://arxiv.org/abs/2303.17760), CAMEL introduces a heuristic tree search approach with critic in the loop, where the expansion and selection are presented below:


<div style={{textAlign: 'center'}}>
    <img src="https://i.imgur.com/6x4ABpp.png" width="600"/>
</div>

To put it simply, a critic agent is a helper agents in the role-playing session, which is capable of selecting proposals and provide informative verbal feedback to the role-playing agents.

## Quick Start

### 🕹 Step 0: Preparations

In [ ]:
%pip install "camel-ai==0.2.16"

In [2]:
from camel.agents import CriticAgent
from camel.generators import SystemMessageGenerator as sys_msg_gen
from camel.messages import BaseMessage as bm
from camel.types import RoleType

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [3]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


Alternatively, if running on Colab, you could save your API keys and tokens as **Colab Secrets**, and use them across notebooks.

To do so, **comment out** the above **manual** API key prompt code block(s), and **uncomment** the following codeblock.

⚠️ Don't forget granting access to the API key you would be using to the current notebook.

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### 🕹 Step 1: Configure the Specifications for Critic Agents

In [4]:
# Set the role name and the task
critic_role = 'a picky critic'

# Create the meta_dict and the role_tuple
meta_dict = dict(critic_role=critic_role,
                 criteria='Help better accomplish the task.')

# Create the role tuple
role_tuple = (critic_role, RoleType.CRITIC)

# Generate the system message
sys_msg = sys_msg_gen().from_dict(meta_dict=meta_dict,
                                  role_tuple=role_tuple)

### 🕹 Step 2: Get the Critic Agents
With the above arguments, we have:

In [5]:
critic_agent = CriticAgent(system_message=sys_msg, verbose=True)

Let's take a look on the default system message:

In [6]:
print(critic_agent.system_message.content)

You are a a picky critic who teams up with a {user_role} and a {assistant_role} to solve a task: {task}.
Your job is to select an option from their proposals and provides your explanations.
Your selection criteria are Help better accomplish the task..
You always have to choose an option from the proposals.


You may overwrite the system message and configure the critic differently based on your own needs.

### 🕹 Step 3: Using Critic Agents for Task Solving
Our `RolePlaying()` class provide a simple way for you to add the critic in the loop. Below we provide a basic pipeline.

In [7]:
# Import necessary classes
from camel.societies import RolePlaying
from camel.configs import ChatGPTConfig
from camel.types import TaskType, ModelType, ModelPlatformType
from colorama import Fore
from camel.utils import print_text_animated
from camel.models import ModelFactory

# Set the LLM model type and model config
model_platform = ModelPlatformType.OPENAI
model_type = ModelType.GPT_4O_MINI
model_config = ChatGPTConfig(
    temperature=0.8,  # the sampling temperature; the higher the more random
    n=3,              # the no. of completion choices to generate for each input
    )

# Create the backend model
model = ModelFactory.create(
    model_platform=model_platform,
    model_type=model_type,
    model_config_dict=model_config.as_dict())

We then need to set the kwargs for the task and each agent:

In [8]:
task_kwargs = {
    'task_prompt': 'Develop a plan to TRAVEL TO THE PAST and make changes.',
    'with_task_specify': True,
    'task_specify_agent_kwargs': {'model': model}
}

user_role_kwargs = {
    'user_role_name': 'an ambitious aspiring TIME TRAVELER',
    'user_agent_kwargs': {'model': model}
}

assistant_role_kwargs = {
    'assistant_role_name': 'the best-ever experimental physicist',
    'assistant_agent_kwargs': {'model': model}
}

critic_role_kwargs = {
    'with_critic_in_the_loop': True,
    'critic_criteria': 'improve the task performance',
    'critic_kwargs': dict(verbose=True)
}

Putting them together:

In [9]:
society = RolePlaying(
    **task_kwargs,             # The task arguments
    **user_role_kwargs,        # The instruction sender's arguments
    **assistant_role_kwargs,   # The instruction receiver's arguments
    **critic_role_kwargs,      # The critic's arguments
)

And the helper functions to run our society:

In [10]:
def is_terminated(response):
    """
    Give alerts when the session should be terminated.
    """
    if response.terminated:
        role = response.msg.role_type.name
        reason = response.info['termination_reasons']
        print(f'AI {role} terminated due to {reason}')

    return response.terminated

In [11]:
def run(society, round_limit: int=10):

    # Get the initial message from the ai assistant to the ai user
    input_msg = society.init_chat()

    # Starting the interactive session
    for _ in range(round_limit):

        # Get the both responses for this round
        assistant_response, user_response = society.step(input_msg)

        # Check the termination condition
        if is_terminated(assistant_response) or is_terminated(user_response):
            break

        # Get the results
        print(f'[AI User] {user_response.msg.content}.\n')
        print(f'[AI Assistant] {assistant_response.msg.content}.\n')

        # Check if the task is end
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            break

        # Get the input message for the next round
        input_msg = assistant_response.msg

    return None

Now let's set our code in motion:

In [12]:
run(society)

> Proposals from an ambitious aspiring TIME TRAVELER (RoleType.USER). Please choose an option:
Option 1:
Instruction: Research the principles of quantum entanglement and wormhole theory.
Input: None

Option 2:
Instruction: Research quantum entanglement and its potential applications in time travel.  
Input: None

Option 3:
Instruction: Research the principles of quantum entanglement and how they can be applied to time travel.  
Input: None

Please first enter your choice ([1-3]) and then your explanation and comparison: 



> Critic response: I choose Option 3: "Research the principles of quantum entanglement and how they can be applied to time travel."

**Explanation and Comparison:**

Option 1 focuses solely on the principles of quantum entanglement and wormhole theory without explicitly linking them to time travel. While understanding these principles is essential, it does not directly address their application in the context of our task, which is to design a time machine.

Option 2

> Proposals from the best-ever experimental physicist (RoleType.ASSISTANT). Please choose an option:
Option 1:
Solution: Quantum entanglement is a phenomenon that occurs when two or more particles become interconnected in such a way that the state of one particle is directly related to the state of the other(s), regardless of the distance between them. This phenomenon is rooted in the principles of quantum mechanics and has intriguing implications for time travel, particularly when combined with wormhole theory.

1. **Understanding Quantum Entanglement**:
   - When two particles are entangled, measuring the state of one particle instantly influences the state of the other, regardless of the distance separating them.
   - This instantaneous correlation suggests that information can be transmitted "faster than light," challenging classical concepts of locality and causality.

2. **Wormhole Theory**:
   - A wormhole is theorized to be a tunnel-like structure in spacetime, connecting dispa

> Proposals from an ambitious aspiring TIME TRAVELER (RoleType.USER). Please choose an option:
Option 1:
Instruction: Identify a pivotal historical event that could be altered and explain the significance of preventing it.  
Input: None

Option 2:
Instruction: Identify a pivotal historical event to alter that could potentially prevent a major war.  
Input: None

Option 3:
Instruction: Identify a pivotal historical event that could be altered to prevent a major war.  
Input: None

Please first enter your choice ([1-3]) and then your explanation and comparison: 



> Critic response: I choose Option 2: "Identify a pivotal historical event to alter that could potentially prevent a major war."

**Explanation and Comparison:**

Option 2 is the most focused and relevant choice for our task of designing a time machine with the intent of altering history. It specifically emphasizes the goal of preventing a major war, which aligns with our overarching objective of minimizing disruption to the t

> Proposals from the best-ever experimental physicist (RoleType.ASSISTANT). Please choose an option:
Option 1:
Solution: A pivotal historical event that could be targeted for alteration to potentially prevent a major war is the assassination of Archduke Franz Ferdinand of Austria on June 28, 1914. This event is widely regarded as the catalyst for World War I, a conflict that resulted in millions of deaths and widespread destruction across Europe.

1. **Event Overview**:
   - Archduke Franz Ferdinand was the heir presumptive to the Austro-Hungarian throne. His assassination by Gavrilo Princip, a member of the nationalist group known as the Black Hand, triggered a series of political and military alliances that led to the outbreak of World War I.

2. **Potential Alteration**:
   - Altering this event could involve preventing the assassination or influencing Ferdinand's decisions and diplomatic approaches in the aftermath. This could help to de-escalate tensions between Austria-Hungary an

> Proposals from an ambitious aspiring TIME TRAVELER (RoleType.USER). Please choose an option:
Option 1:
Instruction: Create a detailed timeline of actions to take for preventing the assassination of Archduke Franz Ferdinand, ensuring minimal disruption to the timeline.  
Input: None

Option 2:
Instruction: Create a detailed timeline of actions to take in order to alter the identified event while ensuring minimal disruption to the timeline.  
Input: None

Option 3:
Instruction: Create a detailed timeline of actions to take for altering the assassination of Archduke Franz Ferdinand, ensuring minimal disruption to the timeline.  
Input: None

Please first enter your choice ([1-3]) and then your explanation and comparison: 



> Critic response: I choose Option 3: "Create a detailed timeline of actions to take for altering the assassination of Archduke Franz Ferdinand, ensuring minimal disruption to the timeline."

**Explanation and Comparison:**

Option 3 is the most focused and relevant

...

> Proposals from an ambitious aspiring TIME TRAVELER (RoleType.USER). Please choose an option:
Option 1:
Instruction: Develop a final summary of the time machine design and the entire plan to alter the assassination of Archduke Franz Ferdinand, including key components and critical actions.  
Input: None

Option 2:
Instruction: Compile a final report summarizing the time machine design, pivotal historical event alteration, timeline of actions, contingency plans, and communication strategy for future reference.  
Input: None

Option 3:
Instruction: Compile a summary of the entire plan for designing a time machine using quantum entanglement and wormhole theory, including the historical event to alter and the strategies developed.  
Input: None

Please first enter your choice ([1-3]) and then your explanation and comparison: 



> Critic response: I choose Option 2: "Compile a final report summarizing the time machine design, pivotal historical event alteration, timeline of actions, conti

> Proposals from the best-ever experimental physicist (RoleType.ASSISTANT). Please choose an option:
Option 1:
Solution: Below is a compiled final report summarizing the design of the time machine, the pivotal historical event selected for alteration, the timeline of actions, contingency plans, and the communication strategy established for future reference.

---

### Final Report: Time Machine Project

**1. Time Machine Design Using Quantum Entanglement and Wormhole Theory**

- **Core Principles**:
  - **Quantum Entanglement**: Exploits the phenomenon where particles become interconnected, allowing for instantaneous information transfer.
  - **Wormhole Theory**: Utilizes theoretical constructs in general relativity to create traversable tunnels in spacetime.

- **Design Components**:
  - **Stable Wormhole Creation**: Hypothetically utilize exotic matter with negative energy density to keep the wormhole open.
  - **Particle Entanglement Setup**: Use techniques such as photon polarizati

> Proposals from an ambitious aspiring TIME TRAVELER (RoleType.USER). Please choose an option:
Option 1:
<CAMEL_TASK_DONE>

Option 2:
<CAMEL_TASK_DONE>

Option 3:
<CAMEL_TASK_DONE>

Please first enter your choice ([1-3]) and then your explanation and comparison: 



> Critic response: It seems that the options provided are placeholders and do not contain specific content to evaluate. Therefore, I cannot choose an option or provide an explanation and comparison without further details. If you could provide the actual content or context for each option, I would be happy to assist you in making a choice and providing an analysis.



> Critic response: It appears that the options provided are still placeholders and do not contain any specific information or tasks to evaluate. Without additional context or details about what each option entails, I am unable to make a choice or provide an explanation and comparison.

If you can provide more information about what each option represents or wh

/usr/local/lib/python3.10/dist-packages/camel/agents/critic_agent.py:144: UserWarning: Critic failed to get a valid option. After 2 attempts. Returning a random option.
  warnings.warn(


> Proposals from the best-ever experimental physicist (RoleType.ASSISTANT). Please choose an option:
Option 1:
Solution: The task has been successfully completed, summarizing all aspects of the time machine design, the pivotal historical event alteration, timeline of actions, contingency plans, and communication strategy. If you require further assistance or wish to explore additional topics related to time travel or any other concept, feel free to ask!

Next request.

Option 2:
Solution: The task is now considered completed. We have successfully designed a time machine using quantum entanglement and wormhole theory, identified a pivotal historical event to alter, created a detailed timeline of actions, prepared contingencies for unforeseen consequences, and established a communication strategy with future self. 

If you need any further assistance or wish to explore new topics or tasks, feel free to ask! 

Next request.

Option 3:
Solution: The task is marked as completed. Thank you f

In this setting, the `AI User` and `AI Assistant` will generate different options when responding (you can simply change the `temperature` in `model_config` to somewhat control the diversity). `AI Critic` will respond with its option selection and reasoning; such additional context will be fed to the two other agents and help them form better subsequent responses.

## Remarks
While we see some performance gains from critic-in-the-loop, it may not really solve the fundamental extrapolation problem (and [self-consistency](https://arxiv.org/abs/2203.11171) remains a strong baseline for many tasks). It is debatable if those agents can extrapolate by self-play within its current scale. A more practical question is how we may *efficiently* introduce *informative* feedbacks/rewards, when agents are connected with external environments and are endowed with tools and memories. They are expected to have a good world model and know how to make abstraction and analogy when necessary. Stay tuned for our next update.